In [74]:
from azure.identity import DefaultAzureCredential, ClientSecretCredential
from azure.storage.blob import (
    BlobServiceClient,
    ContainerClient,
    BlobClient,

    BlobSasPermissions,
    ContainerSasPermissions,
    AccountSasPermissions,
    Services,
    ResourceTypes,
    UserDelegationKey,
    generate_account_sas,
    generate_container_sas,
    generate_blob_sas,
)

from datetime import datetime, timedelta
from argparse import ArgumentParser
from concurrent.futures import ThreadPoolExecutor
from dotenv import load_dotenv
from pathlib import Path
from urllib.parse import urlencode

import requests
import os


In [75]:
# load env variables
env_dir = Path('../').resolve()
load_dotenv(os.path.join(env_dir, '.env'))

True

In [76]:
# Retrieve credentials from environment variables
tenant_id = os.environ.get("AZURE_TENANT_ID")
client_id = os.environ.get("AZURE_CLIENT_ID")
client_secret = os.environ.get("AZURE_CLIENT_SECRET")
subscription_id = os.environ.get("AZURE_SUBSCRIPTION_ID")
storage_account_name = os.environ.get("STORAGE_ACCOUNT_NAME")
storage_account_key = os.environ.get("STORAGE_ACCOUNT_KEY")
resource_group_name = os.environ.get("RESOURCE_GROUP_NAME")

In [77]:
credential = ClientSecretCredential(tenant_id=tenant_id, client_id=client_id, client_secret=client_secret)

# Create a BlobServiceClient object
blob_service_client = BlobServiceClient(account_url=f"https://{storage_account_name}.blob.core.windows.net", credential=credential,)

In [78]:
account_sas_kwargs = {
    "account_name": storage_account_name,
    "account_key": storage_account_key,
    "resource_types": ResourceTypes(
        service=True, 
        container=True, 
        object=True
    ), 
    "permission": AccountSasPermissions(
        read=True, 
        write=True,
        delete=True,
        list=True,
        add=True,
        create=True,
        update=True,
        process=True
    ), 
    "start": datetime.utcnow(),  
    "services": Services(blob=True, queue=True, fileshare=True),
    "expiry": datetime.utcnow() + timedelta(days=1) 
} 

In [79]:
# generated sas token is at the level of the storage account, 
# permitting services like blobs, files, queues, and tables 
# to be read, listed, retrieved, updated, deleted etc. 
# where allowed resource types are service, container 
sas_token = generate_account_sas(**account_sas_kwargs)

In [80]:
# for the sake of example this is an account shared access
# signature (not user delegated shared access signature) 
# generated using the azure portal, when we go at the level
# of our storage account and generate a shared access signature
# token and url

# sas_token = "sv=2024-11-04&ss=bfqt&srt=sco&sp=rwdlacupiytfx&se=2025-07-23T09:46:53Z&st=2025-07-23T01:31:53Z&spr=https&sig=l51AGC01gswW%2FfLY3LpqP8I6W4E%2BdOmmjpr%2FI5LpjWU%3D"
# params = {
#     # signedVersion (required)
#     "sv": "2024-11-04",

#     # signedServices (required)
#     "ss": "bfqt",

#     # signedResourceTypes (required)
#     "srt": "sco",

#     # signedPermissions  (rqeuired)
#     "sp": "rwdlacupiytfx",

#     # signedStart (optional)
#     "se": "2025-07-23T09:46:53Z",

#     # signedEnd (required)
#     "st": "2025-07-23T01:31:53Z",

#     # signedProtocol (optional) default value is "https,http"
#     "spr": "https",

#     # signature (required) is actually a unique string that
#     # results from encoding of base64 result of the SHA256 algorithm
#     # where this algorithm takes in a string-to-sign constructed from 
#     # the param fields like sv, ss, srt, etc. along with a key
#     "sig": "l51AGC01gswW%2FfLY3LpqP8I6W4E%2BdOmmjpr%2FI5LpjWU%3D"
# }

# this is what is basically generated however the problem is we 
# don't exactly know how to generate the hash based authentication
# code (HMAC) signature itself usign the SHA256 algorithm  as this
# is abstracted out during the creation of this sas token in the
# azure portal, and for now I can't find any documentation indicating
# how to use the SHA256 to create this signature from our storage 
# account key and the string-to-sign we can build from our parameters

# but luckily azure python sdk provides a function that generates this 
# sas_token programmatically. And if we wanted access to the whole of
# the storage accounts resources itself, including storage account meta
# data, containers, blobs, etc. we can do so by using generate_account_sas()
# which generates a key at the level of the storage account itself, if we 
# wanted to grant access at the level of the container itself or the individual
# blob file itself we would use generate_container_sas() or generate_blob_sas()
# itself

# we can replicate the whole example sas token above with generate_account_sas()
# which results in the params we want to for instance list all containers and 
# blobs/files in all containers
# st=2025-07-23T02%3A52%3A43Z
# se=2025-07-24T02%3A52%3A43Z
# sp=rwdlacup
# sv=2025-07-05
# ss=bqt
# srt=sco
# sig=pevOTb7Z4Bg%2BHpUgXHRycIxWAlEIYvHdBtUqWuOAauI%3D

In [81]:
blob_service_client = BlobServiceClient(account_url=f"https://{storage_account_name}.blob.core.windows.net", credential=sas_token)
for container in blob_service_client.list_containers():
    print(container.name)
    curr = blob_service_client.get_container_client(container.get("name"))
    for file in curr.list_blobs():
        print(file.name)

sgppipelinesa-bronze
1028-20100710-hne.tgz
1028-20100710-hne/LICENSE
1028-20100710-hne/etc/GPL_license.txt
1028-20100710-hne/etc/HDMan_log
1028-20100710-hne/etc/HVite_log
1028-20100710-hne/etc/PROMPTS
1028-20100710-hne/etc/README
1028-20100710-hne/etc/audiofile_details
1028-20100710-hne/etc/prompts-original
1028-20100710-hne/wav/ar-01.wav
1028-20100710-hne/wav/ar-02.wav
1028-20100710-hne/wav/ar-03.wav
1028-20100710-hne/wav/ar-04.wav
1028-20100710-hne/wav/ar-05.wav
1028-20100710-hne/wav/ar-06.wav
1028-20100710-hne/wav/ar-07.wav
1028-20100710-hne/wav/rp-29.wav
1028-20100710-hne/wav/rp-30.wav
1028-20100710-hne/wav/rp-31.wav
sgppipelinesa-gold
sgppipelinesa-silver
